# Huggingface with ART

In this notebook we will go over how to use the Huggingface AIP with ART. This can enable us to train robust foundation models which act over images. 

Currently this is a developing feature, and so not all ART tools are supported. Further tools and development is planned. As of ART 1.16 we support: 
+ Using a Pytorch backend.
+ Evasion attacks and defences on classical classification tasks such as image classification, but not tasks such as object detection.

If you have a use case that is not supported (or find a bug in this new feature!) please raise an issiue on ART.

Let's look at how we can use ART to secure Huggingface models!


In [1]:
# Relevant imports for the notebook

import transformers
import torch
from torch.optim import Adam

import matplotlib.pyplot as plt
import sys
sys.path.append('../')
import numpy as np
from torchvision import datasets
from art.estimators.hugging_face import HuggingFaceClassifier

In [2]:
# We will use CIFAR data for the notebook.
def get_cifar_data():
    """
    Get CIFAR-10 data.
    :return: cifar train/test data.
    """
    train_set = datasets.CIFAR10('./data', train=True, download=True)
    test_set = datasets.CIFAR10('./data', train=False, download=True)

    x_train = train_set.data.astype(np.float32)
    y_train = np.asarray(train_set.targets)

    x_test = test_set.data.astype(np.float32)
    y_test = np.asarray(test_set.targets)

    x_train = np.moveaxis(x_train, [3], [1])
    x_test = np.moveaxis(x_test, [3], [1])

    x_train = x_train / 255.0
    x_test = x_test / 255.0

    return (x_train, y_train), (x_test, y_test)

In [ ]:
def train_base_model(architecture='google/vit-base-patch16-224'):
    """
    Train a cifar classifier
    """

    (x_train, y_train), (x_test, y_test) = get_cifar_data()

    # Here we load a Huggingface model using the transformers library.
    model = transformers.AutoModelForImageClassification.from_pretrained(architecture,
                                                                         ignore_mismatched_sizes=True,
                                                                         num_labels=10)

    # The HuggingFaceClassifier follows broadly the same API as the PyTorchClassifier
    # So we can supply the loss function, the input shape of the data we will supply, the optimizer, etc.
    # Note, frequently we will be performing fine-tuning or transfer learning with vision transformners and 
    # so we may be fine-tuning on differently sized inputs. 
    # The input_shape argument refers to the shape of the supplied input data which may be different to 
    # the shape required by the model. 
    # To handle this HuggingFaceClassifier has an extra argument of processor which will act on 
    # every batch to process the data into the correct form required by the supplied model.
    # This needs to be manually specified by the user. For many attacks and defences to work it needs to be a 
    # differentiable funciton. 
    # Here the processor is a simple upsampler to enlarge the cifar images into the right size.
    optimizer = Adam(model.parameters(), lr=1e-4)
    upsampler = torch.nn.Upsample(scale_factor=7, mode='nearest')

    hf_model = HuggingFaceClassifier(model, 
                                    loss=torch.nn.CrossEntropyLoss(),
                                    optimizer=optimizer,
                                    input_shape=(3, 32, 32),
                                    nb_classes=10,
                                    processor=upsampler)

    hf_model.fit(x_train, y_train, nb_epochs=2, display_progress_bar=True)
    return hf_model

In [ ]:
hf_base_model = train_base_model()
torch.save(hf_base_model.model.state_dict(), 'hf_base_model.pt')

In [ ]:
def test_pgd(architecture, model_to_test='hf_base_model.pt'):
    """
    Here we can test the model we trained against a PGD attack.
    """
    
    import os
    from art.attacks.evasion import ProjectedGradientDescentPyTorch
    (x_train, y_train), (x_test, y_test) = get_cifar_data()
    model = transformers.AutoModelForImageClassification.from_pretrained(architecture,
                                                                         ignore_mismatched_sizes=True,
                                                                         num_labels=10)
    
    # Load the model state dict from the training loop we just performed.
    model.load_state_dict(torch.load(os.path.join('..', model_to_test)))
    optimizer = Adam(model.parameters(), lr=1e-4)

    # Set it up as a HuggingFaceClassifier
    hf_model = HuggingFaceClassifier(model, 
                                    loss=torch.nn.CrossEntropyLoss(),
                                    optimizer=optimizer,
                                    input_shape=(3, 32, 32),
                                    nb_classes=10,
                                    processor=torch.nn.Upsample(scale_factor=7, mode='nearest'))

    # Let's just use 10 samples for quick demo purposes
    num_samples = 10
    outputs = hf_model(x_test[:num_samples])
    acc = np.sum(np.argmax(outputs.cpu().detach().numpy(), axis=1) == y_test[:num_samples]) / len(y_test[:num_samples])
    print('clean acc ', acc)

    # The backend of the HuggingFaceClassifier is the existing PyTorchClassifier. Thus we can interface HuggingFaceClassifier with
    # aleadry existing attacks in ART which support pytorch. Here we use ProjectedGradientDescentPyTorch.
    attacker = ProjectedGradientDescentPyTorch(hf_model, eps=8/255, eps_step=1/255)
    x_test_adv_robust = attacker.generate(x_test[:num_samples])
    outputs = hf_model(x_test_adv_robust)
    acc = np.sum(np.argmax(outputs.cpu().detach().numpy(), axis=1) == y_test[:num_samples]) / len(y_test[:num_samples])
    print('adv acc ', acc)

    # We can display the adversarial examples to highlight the added perturbation to the original sample.
    x_test_adv_robust = np.moveaxis(x_test_adv_robust, [1], [3])
    x_test = np.moveaxis(x_test, [1], [3])

    delta = x_test[:num_samples] - x_test_adv_robust

    delta = (delta - np.min(delta)) / (np.max(delta) - np.min(delta))
    fig, axs = plt.subplots(3, 3)
    for i in range(3):
        axs[i, 0].imshow(x_test_adv_robust[i])
        axs[i, 1].imshow(x_test[i])
        axs[i, 2].imshow(delta[i])
    plt.tight_layout()

In [ ]:
test_pgd(architecture='google/vit-base-patch16-224', model_to_test='hf_base_model.pt')

In [ ]:
# We can see that we can attack the Huggingface transformer, so now let's use one of the defences in ART!

def adversarial_train():
    from art.defences.trainer import AdversarialTrainerMadryPGD
    (x_train, y_train), (x_test, y_test) = get_cifar_data()
    model = transformers.AutoModelForImageClassification.from_pretrained('google/vit-base-patch16-224',
                                                                         ignore_mismatched_sizes=True,
                                                                         num_labels=10)

    upsampler = torch.nn.Upsample(scale_factor=7, mode='nearest')

    optimizer = Adam(model.parameters(), lr=1e-4)

    hf_model = HuggingFaceClassifier(model, 
                                    loss=torch.nn.CrossEntropyLoss(), 
                                    input_shape=(3, 32, 32),
                                    nb_classes=10,
                                    optimizer=optimizer, 
                                    processor=upsampler)

    # We can now use adversarial training with Madry's protocol.
    trainer = AdversarialTrainerMadryPGD(hf_model,
                                         nb_epochs=10,
                                         eps=8/255,
                                         eps_step=1/255,
                                         max_iter=10)

    trainer.fit(x_train, y_train, display_progress_bar=True)
    torch.save(trainer._classifier.model.state_dict(), 'hf_adv_model.pt')

In [ ]:
# Uncomment the below to run the adverarial training, it can take some time depending on available hardware. 
# The expected runtime is around 15 hours using a Nvidia V100 GPU.

adversarial_train()

In [ ]:
# We now test the adversariallty trained model and we can see we have done from 0% robustness to XXX%.
test_pgd(architecture='google/vit-base-patch16-224', model_to_test='hf_adv_model.pt')

In [ ]:
# We can also try with different models, for example here we try the functions with a different architecture

hf_base_model = train_base_model(architecture='microsoft/swin-tiny-patch4-window7-224')
torch.save(hf_base_model.model.state_dict(), 'swin_tiny_base_model.pt')

test_pgd(architecture='microsoft/swin-tiny-patch4-window7-224', model_to_test='swin_tiny_base_model.pt')

In [ ]:
# We can also try different architectures, for example one of the most popular models on Huggingface is the resn
def train_using_timm_model():
    import timm
    
    model = timm.create_model('resnet50.a1_in1k', pretrained=True)
    upsampler = torch.nn.Upsample(scale_factor=7, mode='nearest')
    
    optimizer = Adam(model.parameters(), lr=1e-3)
    
    hf_model = HuggingFaceClassifier(model, 
                                    loss=torch.nn.CrossEntropyLoss(), 
                                    input_shape=(3, 32, 32),
                                    nb_classes=10,
                                    optimizer=optimizer, 
                                    processor=upsampler)
    (x_train, y_train), (x_test, y_test) = get_cifar_data()
    hf_model.fit(x_train, y_train, nb_epochs=2, display_progress_bar=True)

    return hf_model

In [ ]:
hf_model = train_using_timm_model()
torch.save(hf_base_model.model.state_dict(), 'timm_resnet_50.pt')

In [ ]:
def adversarial_train_timm():
    from art.defences.trainer import AdversarialTrainerMadryPGD
    import timm
    (x_train, y_train), (x_test, y_test) = get_cifar_data()

    model = timm.create_model('resnet50.a1_in1k', pretrained=True)

    upsampler = torch.nn.Upsample(scale_factor=7, mode='nearest')

    optimizer = Adam(model.parameters(), lr=1e-4)

    hf_model = HuggingFaceClassifier(model, 
                                    loss=torch.nn.CrossEntropyLoss(), 
                                    input_shape=(3, 32, 32),
                                    nb_classes=10,
                                    optimizer=optimizer, 
                                    processor=upsampler)

    # We can now use adversarial training with Madry's protocol.
    trainer = AdversarialTrainerMadryPGD(hf_model,
                                         nb_epochs=10,
                                         eps=8/255,
                                         eps_step=1/255,
                                         max_iter=10)

    trainer.fit(x_train, y_train, display_progress_bar=True)
    torch.save(trainer._classifier.model.state_dict(), 'timm_resnet_50_adv_model.pt')

In [ ]:
adversarial_train_timm()

In [ ]:
import torch
from transformers.modeling_utils import PreTrainedModel
from transformers.configuration_utils import PretrainedConfig
from transformers.modeling_outputs import ImageClassifierOutput
from art.estimators.hugging_face import HuggingFaceClassifier

class ModelConfig(PretrainedConfig):
    def __init__(
            self,
            **kwargs,
    ):
        super().__init__(**kwargs)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Model(PreTrainedModel):
    """
    Create model for pytorch.

    The weights and biases are identical to the TensorFlow model in get_classifier_tf().
    """

    def __init__(self, config):
        super().__init__(config)

        self.conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=7)
        self.relu = torch.nn.ReLU()
        self.pool = torch.nn.MaxPool2d(4, 4)
        self.fullyconnected = torch.nn.Linear(25, 10)

    # pylint: disable=W0221
    # disable pylint because of API requirements for function
    def forward(self, x):
        """
        Forward function to evaluate the model
        :param x: Input to the model
        :return: Prediction of the model
        """
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.reshape(-1, 25)
        x = self.fullyconnected(x)
        return ImageClassifierOutput(logits=x)

config = ModelConfig()
pt_model = Model(config=config)
optimizer = torch.optim.Adam(pt_model.parameters(), lr=0.01)

hf_classifier = HuggingFaceClassifier(pt_model,
                                      loss=torch.nn.CrossEntropyLoss(),
                                      optimizer=optimizer,
                                      input_shape=(1, 28, 28),
                                      nb_classes=10,
                                      clip_values=(0, 1),
                                      processor=None)

return hf_classifier